In [1]:
from pipeline import Pipeline
from sklearn.model_selection import train_test_split
import gc


In [2]:
d = Pipeline(model_type="predict_upgrade")
d.concat_df_with_oh_encoding()
df = (
    d.df
)

2022-11-01 09:50:00.670 | INFO     | pipeline:read_all_files:43 - loaded booking data
2022-11-01 09:50:08.921 | INFO     | pipeline:read_all_files:45 - loaded ticket data
2022-11-01 09:50:31.775 | INFO     | pipeline:read_all_files:47 - loaded flight coupon data
2022-11-01 09:50:31.776 | INFO     | pipeline:merge_files:62 - 2022-11-01 09:50:31.776849 merging datasets
2022-11-01 09:50:58.672 | INFO     | pipeline:clean_df:293 - 2022-11-01 09:50:58.672850 cleaning df ...
2022-11-01 09:51:31.880 | INFO     | pipeline:convert_datetime_columns_to_pandas_format:161 - converted datetime columns to pandas format
2022-11-01 09:51:35.151 | INFO     | pipeline:filter_wrong_ticket_prices:144 - filtered wrong ticket prices
2022-11-01 09:51:38.176 | INFO     | pipeline:filter_wrong_booking_window:165 - filtered wrong booking windows
2022-11-01 09:51:41.232 | INFO     | pipeline:filter_nonpositive_flight_distance:169 - filtered nonpositive flight distance
2022-11-01 09:52:51.499 | INFO     | pipeline

In [3]:
df

,COUPON_NUMBER,TIME_DEPARTURE_LOCAL_TIME,FLIGHT_DISTANCE,UPGRADED_FLAG,TOTAL_PRICE_PLN,PAX_GENDER,CORPORATE_CONTRACT_FLG,LOYAL_CUSTOMER,BOOKING_WINDOW_D,STAY_LENGTH_D,...,VAB_STANDARD,PAX_TYPE_CHILD,PAX_TYPE_INFANT,SALES_CHANNEL_ATO_CTO,SALES_CHANNEL_CALL CENTER,SALES_CHANNEL_DCS,SALES_CHANNEL_LOT TRAVEL,SALES_CHANNEL_LOT.COM,TRIP_TYPE_ONE WAY,TRIP_TYPE_ROUND TRIP
0,2,19,1344,0,620.35,1,0,0,121.0,4.0,...,1,0,0,0,0,0,1,0,0,1
1,1,13,6942,0,2151.49,1,0,0,228.0,21.0,...,0,0,0,0,0,0,0,0,0,1
2,2,16,7521,0,2270.18,0,0,0,55.0,15.0,...,0,0,0,0,0,0,0,1,0,1
3,1,6,721,0,1096.16,0,0,0,12.0,7.0,...,1,0,0,0,0,0,0,0,0,1
4,2,18,522,0,752.15,1,0,0,6.0,1.0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9224582,1,19,523,0,813.64,1,0,0,0.0,-9999.0,...,1,0,0,0,0,0,0,0,1,0
9224583,2,17,1070,0,675.51,1,0,0,116.0,8.0,...,0,0,0,0,0,0,0,1,0,1
9224584,1,7,753,0,1408.80,1,0,0,36.0,-9999.0,...,0,0,0,0,0,0,0,0,1,0
9224585,1,18,940,0,1157.97,1,0,0,37.0,3.0,...,0,0,0,0,0,0,0,0,0,1


In [4]:
df.dropna(inplace=True)
X_train, X_test, y_train, y_test = train_test_split(
        df.drop(['UPGRADED_FLAG'], axis=1), df.iloc[:, 3], test_size=0.2, random_state=42
    )
del(df)
gc.collect()

0

In [5]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier(
    scale_pos_weight=1046.6532651902328,
    seed=42,
    use_label_encoder=False,
    tree_method="gpu_hist",
    gpu_id=0,
)
xgb_model.fit(X_train[['FLIGHT_DISTANCE']], y_train, eval_metric="aucpr", verbose=2)


c:\Users\huber\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
c:\Users\huber\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=0, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1046.6532651902328,
              seed=42, subsample=1, tree_method='gpu_hist',
              use_label_encoder=False, validate_parameters=1, verbosity=None)

In [15]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, StratifiedKFold
import warnings
warnings.simplefilter(action='ignore')
kfold2 = StratifiedKFold(n_splits=5)
res = cross_val_score(xgb_model, X_test[['FLIGHT_DISTANCE']], y_test, cv=kfold2, scoring='roc_auc', fit_params={'eval_metric':'aucpr'})
print("średnia:", res.mean())
print('auc score:', res)



średnia: 0.9414075131979891
auc score: [0.93999375 0.93940649 0.94246738 0.9420799  0.94309005]


In [16]:
import numpy as np
results= res
best_result=results.mean()

In [17]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, StratifiedKFold
import xgboost as xgb
import warnings

warnings.simplefilter(action="ignore")
lista_kolumn = ["FLIGHT_DISTANCE"]
wszystkie_kolumny = X_test.columns.tolist()
x = 0
best_col=''
prev_result = results.mean()
while best_col != None:
    best_col = None
    while x < len(wszystkie_kolumny):
        if (wszystkie_kolumny[x] in lista_kolumn):
            x += 1
            continue
        else:
            lista_kolumn.append(wszystkie_kolumny[x])
        kolumny = X_test[lista_kolumn]
        xgb_model = xgb.XGBClassifier(
            scale_pos_weight=1046.6532651902328,
            seed=42,
            use_label_encoder=False,
            tree_method="gpu_hist",
            gpu_id=0,
        )
        kfold = StratifiedKFold(n_splits=5)
        results = cross_val_score(
            xgb_model,
            kolumny,
            y_test,
            cv=kfold,
            scoring="roc_auc",
            fit_params={"eval_metric": "aucpr"},
        )
        # print("średnia:", results.mean())
        # print("auc score:", results)
        # print("kolumny:", kolumny.columns)
        if best_result < results.mean():
            best_col = wszystkie_kolumny[x]
            best_result = results.mean()
        lista_kolumn.remove(wszystkie_kolumny[x])
        x += 1
    if best_col: 
        print("best column: ", best_col, best_result)
        print("AUC score increase: ", best_result - prev_result)
        lista_kolumn.append(best_col)
        print("Current column list: ", lista_kolumn)
        x = 0
        prev_result = best_result


best column:  BOOKED_CABIN_Premium 0.9499088278900393
AUC score increase:  0.008501314692050177
Current column list:  ['FLIGHT_DISTANCE', 'BOOKED_CABIN_Premium']
best column:  if_additional_upgrade 0.9525561734449081
AUC score increase:  0.002647345554868785
Current column list:  ['FLIGHT_DISTANCE', 'BOOKED_CABIN_Premium', 'if_additional_upgrade']
best column:  BOOKED_CABIN_Economy 0.95425278352564
AUC score increase:  0.001696610080731964
Current column list:  ['FLIGHT_DISTANCE', 'BOOKED_CABIN_Premium', 'if_additional_upgrade', 'BOOKED_CABIN_Economy']
best column:  VAB_SAVER 0.9551517114332249
AUC score increase:  0.0008989279075848344
Current column list:  ['FLIGHT_DISTANCE', 'BOOKED_CABIN_Premium', 'if_additional_upgrade', 'BOOKED_CABIN_Economy', 'VAB_SAVER']
best column:  VAB_STANDARD 0.9559176013552427
AUC score increase:  0.0007658899220178705
Current column list:  ['FLIGHT_DISTANCE', 'BOOKED_CABIN_Premium', 'if_additional_upgrade', 'BOOKED_CABIN_Economy', 'VAB_SAVER', 'VAB_STAND